In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
query_string = """
select
    aliquot_barcode,
    HGNC_gene_symbol as gene_name,
    sum(normalized_count) as counts_hg19
from `gdc_tcga_skcm_subset.RNAseq_hg19_gdc_current`
where aliquot_barcode in (
    select aliquot_barcode 
    from `gdc_tcga_skcm_subset.aliquot2caseIDmap_current`
    where sample_type_name = "Metastatic"
)
group by 1, 2
order by 1, 2
"""

query_job = client.query(query_string)
tcga_skcm_hg19 = query_job.to_dataframe(progress_bar_type="tqdm")

Query complete after 29.06s: : 3query [00:28,  9.64s/query]                                                    
Downloading: 100%|██████████| 7407161/7407161 [00:04<00:00, 1801568.72rows/s]


In [4]:
query_string = """
select
    aliquot_barcode,
    gene_name,
    sum(HTSeq__Counts) as counts_hg38,
    sum(HTSeq__FPKM) as fpkm_hg38,
    sum(HTSeq__FPKM_UQ) as fpkm_uq_hg38
from `gdc_tcga_skcm_subset.RNAseq_hg38_gdc_current`
where aliquot_barcode in (
    select aliquot_barcode 
    from `gdc_tcga_skcm_subset.aliquot2caseIDmap_current`
    where sample_type_name = "Metastatic"
)
group by 1, 2
order by 1, 2
"""

query_job = client.query(query_string)
tcga_skcm_hg38 = query_job.to_dataframe(progress_bar_type="tqdm")

Query complete after 22.50s: : 3query [00:22,  7.50s/query]                                                    
Downloading: 100%|██████████| 7234304/7234304 [00:04<00:00, 1702711.96rows/s]


In [ ]:
# both = pd.concat([tcga_skcm_hg19.set_index("HGNC_gene_symbol"), tcga_skcm_hg38.set_index("gene_name")], axis=1, join="inner")
tcga_skcm_hg19.set_index(["aliquot_barcode", "gene_name"])
tcga_skcm_hg38.set_index(["aliquot_barcode", "gene_name"])

In [ ]:
# both.isna().sum()

In [7]:
pd.merge(tcga_skcm_hg19, tcga_skcm_hg38)  # , on=["aliquot_barcode", "gene_name"])

,aliquot_barcode,gene_name,counts_hg19,counts_hg38,fpkm_hg38,fpkm_uq_hg38
0,TCGA-3N-A9WB-06A-11R-A38C-07,A1BG,381.0662,68,0.316945,7.772224e+03
1,TCGA-3N-A9WB-06A-11R-A38C-07,A1CF,0.0000,0,0.000000,0.000000e+00
2,TCGA-3N-A9WB-06A-11R-A38C-07,A2M,2209.5194,5928,17.338106,4.251701e+05
3,TCGA-3N-A9WB-06A-11R-A38C-07,A2ML1,7.2698,34,0.086929,2.131695e+03
4,TCGA-3N-A9WB-06A-11R-A38C-07,A4GALT,33.5218,91,0.577348,1.415789e+04
...,...,...,...,...,...,...
6743179,TCGA-Z2-AA3V-06A-11R-A39D-07,ZYG11A,0.0000,0,0.000000,0.000000e+00
6743180,TCGA-Z2-AA3V-06A-11R-A39D-07,ZYG11B,75.8542,130,0.344766,1.453485e+04
6743181,TCGA-Z2-AA3V-06A-11R-A39D-07,ZYX,11832.5977,18916,118.110885,4.979386e+06
6743182,TCGA-Z2-AA3V-06A-11R-A39D-07,ZZEF1,1413.1110,2265,3.101429,1.307518e+05


In [ ]:
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io

# plotly.io.renderers.default = "jupyterlab+png"

In [ ]:
fig = px.scatter(both, x="HTSeq__Counts_mean", y="normalized_count_mean", trendline="ols")

In [ ]:
fig

In [ ]:
fig.show(height=500, scale=2, renderer="png")

In [ ]:
fig = px.scatter(both, x="HTSeq__Counts_mean", y="normalized_count_mean", log_x=True, log_y=True)
fig.update_traces(marker_size=1)

In [ ]:
fig.show(height=600, scale=2, renderer="png")